In [2]:
import numpy as np
import math
import itertools
import picos as pic
import mosek

In [3]:
def K_fun(m,k,n):

    Fun=0

    for q in range(0,m+1):
        Fun+= math.comb(n-k,m-q)*math.comb(k,q)*(-1)**q
    return Fun 

In [8]:
def SDP(n,K,d,p,D):
    
    ##### SDP #####

    T=2**p
    
    pic.available_solvers()
    
    
    Ap = pic.RealVariable("Ap", n+1)
    
    sdp = pic.Problem()

    sdp.set_objective('find')
    
    L=K**4
    
    sdp.add_constraint(Ap[0]==L)
    sdp.add_constraint(Ap[n]==L/K)
    
    sdp.add_list_of_constraints((K*Ap[n-i]-Ap[i])== 0 for i in range(d)) 

    for m in range(n+1):
        A=0
        for j in range(m+1):
            A+=(-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*Ap[j]
        sdp.add_constraint(A>=0)            
    
    
    ### K*B_j-A_j>=0 ###
    for m in range(n+1):   
                                    
        sdp.add_constraint(                              
        sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1)) >= 0)
    
    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        sdp.add_constraint(sum(K_fun(n-j,i,n)*Ap[i] for i in range(n+1)) >= 0)
    
    
    #print(sdp.variables)
    
    sol = sdp.solve(solver='mosek', primals=False)

    
    #pic.solvers.solver_mosek()

    if sol.problemStatus == "feasible":
        return True                    
    elif sol.problemStatus == "infeasible":
        
        dual=sdp.dual

        dual.solve(solver='mosek', duals=False)

        return False, "{:.2e}".format(dual.objective.value)
    else:
        return sol.problemStatus 

    
D=2

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],
]

Codes=[
[1,0,1],[1,1,1],
[2,0,2],
[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
#[4,0,2],
#[4,1,2],
#[4,2,2],[4,3,1],[4,4,1],
[5,0,3],
#[5,1,3],
[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],
[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],
[8,0,4],[8,1,3],[8,2,3],[8,3,3],[8,4,2],[8,5,2],[8,6,2],[8,7,1],[8,8,1],
[9,0,4],[9,1,3],[9,2,3],[9,3,3],[9,4,2],[9,5,2],[9,6,2],[9,7,1],[9,8,1],[9,9,1],
[10,0,4],[10,1,4],[10,2,4],[10,3,3],[10,4,3],[10,5,2],[10,6,2],[10,7,2],[10,8,2],[10,9,1],[10,10,1]
]

#Codes=[[6,2,2]]
#Codes=[[7,7,1]]
#Codes=[[6,6,1]]
#Codes=[[10, 8, 3]]
#Codes=[[2,0,2]]
#### Existing qubit codes ####


for code in Codes:
    
    n,K,d=code
    p=n
    check=SDP(n,2**K,d,p,D)
    
    if check != True:
        print("%s=%s" % (code,check))

print("Done")



#### Not existing qubit codes (distance+1) ####

for code in Codes:
    
    n,K,d=code
    p=n
    check=SDP(n,2**K,d+1,p,D)
    
    #if check != False:
    print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))



[6, 6, 1]=(False, '1.00e+00')
[8, 8, 1]=unknown
[9, 8, 1]=(False, '1.00e+00')
[9, 9, 1]=(False, '-5.56e+13')
[10, 8, 2]=unknown
[10, 9, 1]=(False, '1.00e+00')
[10, 10, 1]=(False, '-1.02e+20')
Done
[1,0,2]=True
[1,1,2]=(False, '2.40e+01')
[2,0,3]=True
[2,1,2]=(False, '3.20e+01')
[2,2,2]=(False, '5.12e+02')
[3,0,3]=True
[3,1,2]=(False, '4.00e+00')
[3,2,2]=(False, '3.84e+02')
[3,3,2]=(False, '9.22e+03')
[5,0,4]=True
[5,2,3]=(False, '1.00e+00')
[5,3,2]=(False, '1.00e+00')
[5,4,2]=(False, '1.00e+00')
[5,5,2]=(False, '1.00e+00')
[6,0,5]=True
[6,1,4]=(False, '1.00e+00')
[6,2,3]=(False, '1.00e+00')
[6,3,3]=(False, '1.00e+00')
[6,4,3]=(False, '1.00e+00')
[6,5,2]=(False, '1.00e+00')
[6,6,2]=(False, '1.00e+00')
[7,0,4]=True
[7,1,4]=(False, '1.00e+00')
[7,2,3]=(False, '1.00e+00')
[7,3,3]=(False, '1.00e+00')
[7,4,3]=(False, '1.00e+00')
[7,5,2]=(False, '1.00e+00')
[7,6,2]=(False, '1.00e+00')
[7,7,2]=(False, '1.00e+00')
[8,0,5]=True
[8,1,4]=(False, '1.00e+00')
[8,2,4]=(False, '1.00e+00')
[8,3,4]=(Fal